In [1]:
import os
import sys
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchmetrics.classification import MultilabelF1Score, MultilabelAccuracy
from tqdm.auto import tqdm
import networkx as nx       # Dùng cho GOPropagator
from collections import defaultdict # Dùng cho NegativePropagator

# Tắt cảnh báo không cần thiết (tùy chọn)
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


# CONFIG

In [2]:
class Config:
    # --- Control Flags ---
    IS_TRAINING = False       # Set True để train lại 3 model, False để nộp bài
    FORCE_RERUN_PREDICT = False 
    
    # --- Paths ---
    ESM_DIR = '/kaggle/input/cafa6-protein-embeddings-esm2'
    TRAIN_TERMS = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv'
    TEST_FASTA = '/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta'
    GO_OBO = '/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo'
    GOA_FILE = '/kaggle/input/protein-go-annotations/goa_uniprot_all.csv'
    HOMOLOGY_FILE = '/kaggle/input/foldseek-cafa/foldseek_submission.tsv'
    
    # --- Model Checkpoints (Multi-Ontology) ---
    # Thay đổi path này trỏ đến dataset chứa 3 model sau khi bạn train xong
    CKPT_BPO = '/kaggle/input/multi-ontology/pytorch/default/1/model_BPO.pth'
    CKPT_CCO = '/kaggle/input/multi-ontology/pytorch/default/1/model_CCO.pth'
    CKPT_MFO = '/kaggle/input/multi-ontology/pytorch/default/1/model_MFO.pth'
    
    # --- Model Configs (Customize per Aspect) ---
    EMBED_DIM = 1280
    BATCH_SIZE = 128
    LR = 1e-3
    EPOCHS = 10
    
    # Số lượng nhãn cho từng loại (BPO thường nhiều nhãn hơn)
    NUM_LABELS_BPO = 1500
    NUM_LABELS_CCO = 800
    NUM_LABELS_MFO = 800
    
    # --- Post-processing ---
    THRESHOLD = 0.005 # Hạ thấp threshold một chút vì chúng ta sẽ lọc kỹ hơn
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"⚙️ Running on device: {Config.DEVICE}")

⚙️ Running on device: cpu


In [3]:
def get_term_aspect_map(obo_path):
    """Hàm đọc file OBO để biết GO Term nào thuộc nhóm nào (BPO/CCO/MFO)"""
    print(f"📚 Parsing Aspect (Namespace) from {obo_path}...")
    term_to_aspect = {}
    current_term = None
    
    # Map namespace trong OBO sang tên viết tắt chuẩn
    ns_map = {
        'biological_process': 'BPO',
        'cellular_component': 'CCO',
        'molecular_function': 'MFO'
    }
    
    with open(obo_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith("[Term]"):
                current_term = None
            elif line.startswith("id: GO:"):
                current_term = line[4:]
            elif line.startswith("namespace:") and current_term:
                ns = line.split()[1]
                if ns in ns_map:
                    term_to_aspect[current_term] = ns_map[ns]
    
    print(f"   Mapped {len(term_to_aspect)} terms to aspects.")
    return term_to_aspect

# DATASET

In [4]:
class ProteinDataset(Dataset):
    def __init__(self, embeddings, labels=None):
        """
        embeddings: numpy array hoặc tensor (N_samples, Embed_Dim)
        labels: numpy array hoặc tensor (N_samples, Num_Labels) - Optional
        """
        # Chuyển đổi sang FloatTensor ngay từ đầu để đỡ tốn công trong loop
        self.embeddings = torch.tensor(embeddings, dtype=torch.float32)
        
        if labels is not None:
            self.labels = torch.tensor(labels, dtype=torch.float32)
        else:
            self.labels = None
            
    def __len__(self):
        return len(self.embeddings)
    
    def __getitem__(self, idx):
        # Trả về cặp (X, y) hoặc (X)
        if self.labels is not None:
            return self.embeddings[idx], self.labels[idx]
        return self.embeddings[idx]

# MODEL

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features, hidden_features, dropout=0.3):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.BatchNorm1d(hidden_features),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_features, in_features),
            nn.BatchNorm1d(in_features)
        )
        self.relu = nn.ReLU()
    def forward(self, x): return self.relu(x + self.block(x))

class ProteinClassifier(nn.Module):
    def __init__(self, input_dim=1280, num_classes=1500):
        super().__init__()
        self.bn_input = nn.BatchNorm1d(input_dim)
        self.layer1 = nn.Linear(input_dim, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.res_block = ResidualBlock(512, 256)
        self.out = nn.Linear(512, num_classes)
    def forward(self, x):
        x = self.bn_input(x)
        x = self.dropout(self.relu(self.layer1(x)))
        x = self.res_block(x)
        return self.out(x)

# DATA MANAGER

In [6]:
class CAFA6DataManager:
    def __init__(self, config):
        self.cfg = config
        self.term_to_idx = {} 
        self.pid_to_emb_idx = {}
        self.all_embeds = None
        self.test_ids = []
        # Load map aspect ngay khi khởi tạo
        self.term_aspect_map = get_term_aspect_map(config.GO_OBO)

    def _load_safe_ids(self, path):
        try:
            ids = np.load(path, allow_pickle=True)
            if len(ids) > 0 and isinstance(ids[0], (bytes, np.bytes_)):
                return [i.decode('utf-8') for i in ids]
            return ids.tolist()
        except: return []

    def load_embeddings(self):
        print("📥 Loading Embeddings source...")
        try: 
            all_pids = pd.read_csv(os.path.join(self.cfg.ESM_DIR, "protein_ids.csv"))['protein_id'].tolist()
        except: 
            all_pids = self._load_safe_ids(os.path.join(self.cfg.ESM_DIR, "protein_ids.npy"))
            
        self.all_embeds = np.load(os.path.join(self.cfg.ESM_DIR, "protein_embeddings.npy"), mmap_mode='r')
        self.pid_to_emb_idx = {pid: i for i, pid in enumerate(all_pids)}
        print(f"   Loaded {len(all_pids)} protein embeddings.")

    def _get_labels_matrix(self, valid_pids, target_aspect, num_labels):
        """Tạo matrix Y chỉ chứa các nhãn thuộc Aspect đang train"""
        print(f"   Processing Labels Matrix for aspect: {target_aspect}...")
        df = pd.read_csv(self.cfg.TRAIN_TERMS, sep="\t")
        
        # 1. Map term sang aspect và Lọc
        df['aspect'] = df['term'].map(self.term_aspect_map)
        df = df[df['aspect'] == target_aspect]
        
        if df.empty:
            raise ValueError(f"Không tìm thấy dữ liệu cho aspect {target_aspect}. Kiểm tra OBO parser.")

        # 2. Lấy Top Labels của Aspect đó
        top_terms = df['term'].value_counts().index[:num_labels].tolist()
        self.term_to_idx = {t: i for i, t in enumerate(top_terms)}
        
        # 3. Tạo Matrix Y
        pid_to_idx = {pid: i for i, pid in enumerate(valid_pids)}
        y = np.zeros((len(valid_pids), num_labels), dtype=np.float32)
        
        df = df[df['term'].isin(top_terms) & df['EntryID'].isin(valid_pids)]
        row_idx = df['EntryID'].map(pid_to_idx)
        col_idx = df['term'].map(self.term_to_idx)
        valid = row_idx.notna() & col_idx.notna()
        
        y[row_idx[valid].astype(int), col_idx[valid].astype(int)] = 1.0
        return y

    def prepare_train_loaders(self, aspect, num_labels):
        """Chuẩn bị DataLoader riêng cho từng Aspect"""
        if self.all_embeds is None: self.load_embeddings()

        print(f"🛠️ Preparing Training Data for [{aspect}]...")
        
        # Filter Valid Train IDs
        df_terms = pd.read_csv(self.cfg.TRAIN_TERMS, sep="\t", usecols=['EntryID'])
        train_targets = set(df_terms['EntryID'].unique())
        valid_pids = [p for p in train_targets if p in self.pid_to_emb_idx]
        
        # Construct X
        train_emb_indices = [self.pid_to_emb_idx[p] for p in valid_pids]
        X = np.array([self.all_embeds[i] for i in train_emb_indices])
        
        # Construct y (theo aspect)
        y = self._get_labels_matrix(valid_pids, aspect, num_labels)
        
        # Split & Loaders
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
        
        train_ds = ProteinDataset(X_train, y_train)
        val_ds = ProteinDataset(X_val, y_val)
        
        train_dl = DataLoader(train_ds, batch_size=self.cfg.BATCH_SIZE, shuffle=True, num_workers=0)
        val_dl = DataLoader(val_ds, batch_size=self.cfg.BATCH_SIZE, shuffle=False, num_workers=0)
        
        del X, y, X_train, y_train, X_val, y_val
        gc.collect()
        return train_dl, val_dl
    
    def prepare_test_loader(self):
        # Giữ nguyên code cũ
        if self.all_embeds is None: self.load_embeddings()

        print(f"📖 Reading Test Fasta: {self.cfg.TEST_FASTA}")
        self.test_ids = []
        with open(self.cfg.TEST_FASTA, 'r') as f:
            for line in f:
                if line.startswith('>'): self.test_ids.append(line.strip()[1:].split()[0])
        
        print("   Constructing Test Matrix...")
        X_test = np.zeros((len(self.test_ids), self.cfg.EMBED_DIM), dtype=np.float32)
        
        found = 0
        for i, pid in enumerate(self.test_ids):
            if pid in self.pid_to_emb_idx:
                X_test[i] = self.all_embeds[self.pid_to_emb_idx[pid]]
                found += 1
        print(f"   Found vectors for {found}/{len(self.test_ids)} test proteins.")     
        test_dl = DataLoader(ProteinDataset(X_test), batch_size=self.cfg.BATCH_SIZE, shuffle=False, num_workers=0)
        return test_dl

# TRAINER 

In [7]:
class Trainer:
    def __init__(self, model, train_loader, val_loader, config):
        self.model = model
        self.train_dl = train_loader
        self.val_dl = val_loader
        self.cfg = config
        
        # --- FIX: LẤY SỐ LABEL TỪ OUTPUT LAYER CỦA MODEL ---
        # model.out là lớp Linear cuối cùng: nn.Linear(512, num_classes)
        self.num_labels = model.out.out_features 
        
        self.optimizer = torch.optim.Adam(model.parameters(), lr=config.LR)
        self.criterion = nn.BCEWithLogitsLoss()
        
        # Metrics: Dùng self.num_labels thay vì config.NUM_LABELS
        self.f1_score = MultilabelF1Score(num_labels=self.num_labels, average='micro').to(config.DEVICE)
        self.acc_score = MultilabelAccuracy(num_labels=self.num_labels, average='micro').to(config.DEVICE)

    def train_epoch(self, epoch_idx):
        self.model.train()
        total_loss = 0
        self.f1_score.reset()
        self.acc_score.reset()
        
        for X, y in self.train_dl:
            X, y = X.to(self.cfg.DEVICE), y.to(self.cfg.DEVICE)
            
            self.optimizer.zero_grad()
            out = self.model(X)
            loss = self.criterion(out, y)
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            self.f1_score.update(out, y)
            self.acc_score.update(out, y)
            
        avg_loss = total_loss / len(self.train_dl)
        f1 = self.f1_score.compute()
        acc = self.acc_score.compute()
        print(f"   Epoch {epoch_idx}/{self.cfg.EPOCHS} | Loss: {avg_loss:.4f} | Train F1: {f1:.4f} | Acc: {acc:.4f}")

    def validate(self):
        self.model.eval()
        self.f1_score.reset()
        with torch.no_grad():
            for X, y in self.val_dl:
                X, y = X.to(self.cfg.DEVICE), y.to(self.cfg.DEVICE)
                out = self.model(X)
                self.f1_score.update(out, y)
        print(f"   🔥 Validation F1: {self.f1_score.compute():.4f}")

    def fit(self):
        print("🚀 Start Training...")
        for epoch in range(1, self.cfg.EPOCHS + 1):
            self.train_epoch(epoch)
        self.validate()
        print("✅ Training Completed.")

    def predict(self, test_dl):
        print("🔮 Predicting...")
        self.model.eval()
        preds = []
        with torch.no_grad():
            for X in tqdm(test_dl):
                X = X.to(self.cfg.DEVICE)
                preds.append(torch.sigmoid(self.model(X)).cpu().numpy())
        return np.vstack(preds)

# GO PROPAGATOR

In [8]:
import networkx as nx
class GOPropagator:
    def __init__(self, obo_path, term_to_idx):
        self.obo_path = obo_path
        self.term_to_idx = term_to_idx
        self.idx_to_term = {v: k for k, v in term_to_idx.items()}
        self.graph = None
        
    def load_obo(self):
        print(f"🌳 Parsing OBO structure from: {self.obo_path}")
        self.graph = nx.DiGraph()
        
        current_term = None
        with open(self.obo_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith("[Term]"):
                    current_term = None
                elif line.startswith("id: GO:"):
                    current_term = line[4:]
                    self.graph.add_node(current_term)
                elif line.startswith("is_a: GO:") and current_term:
                    parent = line.split()[1]
                    # Thêm cạnh từ Con -> Cha (để dễ duyệt ngược lên)
                    self.graph.add_edge(current_term, parent)
                    
    def propagate(self, probs_matrix):
        """
        Cập nhật điểm số theo quy tắc: Điểm cha = max(Điểm cha, Điểm con)
        """
        if self.graph is None: self.load_obo()
        
        print("🌊 Starting Score Propagation (Bottom-up)...")
        
        # 1. Lọc ra các GO terms có trong mô hình của chúng ta
        # Sắp xếp theo thứ tự topo (từ lá lên gốc) để lan truyền hiệu quả nhất
        # Tuy nhiên, chỉ cần đảm bảo duyệt qua tất cả quan hệ
        
        # Lấy danh sách index các term có trong mô hình
        valid_terms = set(self.term_to_idx.keys())
        
        # Tìm các cạnh quan hệ mà cả Cha và Con đều nằm trong tập nhãn dự đoán
        relevant_edges = []
        for child, parent in self.graph.edges():
            if child in valid_terms and parent in valid_terms:
                relevant_edges.append((self.term_to_idx[child], self.term_to_idx[parent]))
        
        print(f"   Found {len(relevant_edges)} parent-child relationships within the {len(valid_terms)} predicted labels.")
        
        # 2. Thực thi lan truyền
        # Do cấu trúc DAG phức tạp, ta lặp vài lần để đảm bảo điểm lan truyền hết từ đáy lên đỉnh
        # (Hoặc dùng topological sort nhưng lặp 3-4 lần là đủ cho độ sâu GO)
        new_probs = probs_matrix.copy()
        
        for i in range(3): # Lặp 3 lần để lan truyền sâu
            count_updates = 0
            for child_idx, parent_idx in relevant_edges:
                # Quy tắc: Score cha = Max(Score cha, Score con)
                # Dùng np.maximum để tính toán song song trên toàn bộ batch proteins
                original_parent_scores = new_probs[:, parent_idx]
                updated_scores = np.maximum(new_probs[:, parent_idx], new_probs[:, child_idx])
                
                # Kiểm tra xem có thay đổi không (optional, để debug)
                # new_probs[:, parent_idx] = updated_scores
                
                # Optimization: Chỉ gán lại
                new_probs[:, parent_idx] = updated_scores
                
            print(f"   Iteration {i+1} complete.")
            
        return new_probs


# NEGATIVE PROPAGATOR

In [9]:
class NegativePropagator:
    def __init__(self, config):
        self.cfg = config
        self.children = defaultdict(list)
        self.negative_pairs = set()

    def parse_obo(self):
        print(f"🌳 Parsing OBO: {self.cfg.GO_OBO}") # <--- Dùng GO_OBO
        term = None
        with open(self.cfg.GO_OBO, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('[Term]'): term = None
                elif line.startswith('id: GO:'): term = line[4:]
                elif line.startswith('is_a: GO:') and term:
                    self.children[line.split()[1]].append(term)

    def _get_descendants(self, root):
        desc = set()
        stack = [root]
        while stack:
            curr = stack.pop()
            if curr in self.children:
                for kid in self.children[curr]:
                    if kid not in desc:
                        desc.add(kid)
                        stack.append(kid)
        return desc

    def load_negatives(self, target_ids):
        if not os.path.exists(self.cfg.GOA_FILE): # <--- Dùng GOA_FILE
            print("⚠️ GOA File not found. Skipping.")
            return

        print("🚫 Loading Negative Annotations...")
        target_set = set(target_ids)
        chunk_iter = pd.read_csv(self.cfg.GOA_FILE, chunksize=500000, usecols=['protein_id', 'go_term', 'qualifier'])
        
        for chunk in chunk_iter:
            neg = chunk[(chunk['qualifier'].str.contains('NOT', na=False)) & (chunk['protein_id'].isin(target_set))]
            for _, row in neg.iterrows():
                pid, term = row['protein_id'], row['go_term']
                self.negative_pairs.add(f"{pid}_{term}")
                for d in self._get_descendants(term):
                    self.negative_pairs.add(f"{pid}_{d}")
        print(f"   Found {len(self.negative_pairs)} negative constraints.")

# MAIN EXECUTION

In [10]:
def load_specialist_model(ckpt_path, num_labels):
    print(f"📥 Loading specialist model from {ckpt_path}...")
    checkpoint = torch.load(ckpt_path, map_location=Config.DEVICE)
    
    # Init Model
    model = ProteinClassifier(Config.EMBED_DIM, num_labels).to(Config.DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    return model, checkpoint['term_to_idx']

In [11]:
def get_weighted_thresholds(term_to_idx, train_terms_path, min_t=0.01, max_t=0.15):
    """
    Tính ngưỡng động dựa trên tần suất xuất hiện của nhãn trong tập Train.
    - Nhãn xuất hiện nhiều -> Threshold cao (max_t)
    - Nhãn xuất hiện ít -> Threshold thấp (min_t)
    """
    print("⚖️ Calculating Weighted Thresholds based on Term Frequency...")
    
    # 1. Đếm số lần xuất hiện của từng term trong tập Train
    df = pd.read_csv(train_terms_path, sep="\t", usecols=['term'])
    term_counts = df['term'].value_counts()
    
    # 2. Khởi tạo vector threshold (mặc định là min_t)
    num_labels = len(term_to_idx)
    thresholds = np.full(num_labels, min_t, dtype=np.float32)
    
    # 3. Tính toán threshold cho từng nhãn
    max_count = term_counts.iloc[0] # Số lượng của nhãn phổ biến nhất
    
    found_count = 0
    for term, idx in term_to_idx.items():
        if term in term_counts.index:
            count = term_counts[term]
            # Công thức nội suy tuyến tính:
            # T_i = Min + (Max - Min) * (Count_i / Max_Count)
            freq_ratio = count / max_count
            thresholds[idx] = min_t + (max_t - min_t) * freq_ratio
            found_count += 1
            
    print(f"   Calculated thresholds for {found_count}/{num_labels} terms.")
    print(f"   Range: {thresholds.min():.4f} -> {thresholds.max():.4f}")
    
    return thresholds

In [12]:
def load_foldseek_matrix(foldseek_path, target_pids, term_to_idx, shape):
    """
    Đọc file Foldseek và chuyển thành ma trận numpy [N_test, N_labels]
    khớp với thứ tự của model Deep Learning.
    """
    print(f"🧬 Loading Foldseek results from: {foldseek_path}")
    
    # 1. Tạo ma trận rỗng
    fs_matrix = np.zeros(shape, dtype=np.float32)
    
    # 2. Tạo map nhanh cho Protein ID (ID -> Row Index)
    pid_to_row = {pid: i for i, pid in enumerate(target_pids)}
    
    # 3. Đọc file Foldseek (TSV: EntryID, term, score)
    # Lưu ý: File Foldseek thường rất nặng, cần đọc tối ưu
    try:
        # Giả sử file không có header, cột 0 là ID, 1 là Term, 2 là Score
        df = pd.read_csv(foldseek_path, sep='\t', header=None, names=['EntryID', 'term', 'score'], 
                         dtype={'score': np.float32}, usecols=[0, 1, 2])
        
        # 4. Lọc dữ liệu
        # Chỉ lấy những dòng có ID nằm trong Test Set và Term nằm trong Top 1500 của mình
        df = df[df['EntryID'].isin(pid_to_row) & df['term'].isin(term_to_idx)]
        
        print(f"   Mapping {len(df)} Foldseek predictions to matrix...")
        
        # 5. Điền vào ma trận (Vectorized operation sẽ nhanh hơn loop)
        # Map ID và Term sang index
        row_indices = df['EntryID'].map(pid_to_row).values
        col_indices = df['term'].map(term_to_idx).values
        scores = df['score'].values
        
        # Gán điểm (nếu có trùng lặp, dòng sau đè dòng trước - hoặc dùng hàm aggregate nếu cần)
        fs_matrix[row_indices, col_indices] = scores
        
        print("   Foldseek matrix constructed successfully.")
        return fs_matrix
        
    except Exception as e:
        print(f"⚠️ Error loading Foldseek: {e}")
        print("   Returning empty matrix (only DL model will be used).")
        return np.zeros(shape, dtype=np.float32)

def blend_predictions(dl_probs, fs_probs):
    """
    Chiến thuật: Ưu tiên DL, chỉ dùng Foldseek nếu DL = 0
    """
    # Copy ma trận DL
    final_probs = dl_probs.copy()
    
    # Tìm những chỗ mà DL model hoàn toàn không biết gì (score xấp xỉ 0)
    # Nhưng Foldseek lại tìm thấy manh mối
    mask = (dl_probs < 0.001) & (fs_probs > 0.0)
    
    # Điền điểm Foldseek vào những chỗ đó
    final_probs[mask] = fs_probs[mask]
    
    return final_probs

In [13]:
if __name__ == "__main__":
    
    # --- ⚠️ FIX PATHS HERE IF YOU HAVE TRAINED MODELS ---
    # Ensure these point to the correct files (e.g., model_CCO.pth, model_MFO.pth)
    # If you haven't trained CCO/MFO yet, this code will safely skip them if they fail to load.
    aspects_config = [
        ('BPO', Config.NUM_LABELS_BPO, 'model_BPO.pth' if Config.IS_TRAINING else Config.CKPT_BPO),
        ('CCO', Config.NUM_LABELS_CCO, 'model_CCO.pth' if Config.IS_TRAINING else Config.CKPT_CCO),
        ('MFO', Config.NUM_LABELS_MFO, 'model_MFO.pth' if Config.IS_TRAINING else Config.CKPT_MFO)
    ]

    # ==========================\n    # PHASE 1: TRAINING\n    # ==========================
    if Config.IS_TRAINING:
        print("🚀 STARTING MULTI-ONTOLOGY TRAINING PIPELINE...")
        
        dm = CAFA6DataManager(Config)
        dm.load_embeddings()
        
        for aspect, num_lbls, save_path in aspects_config:
            print(f"\n{'='*40}")
            print(f" >>> Training Specialist Model: {aspect} ({num_lbls} labels) <<<")
            print(f"{'='*40}")
            
            try:
                # 1. Prepare Data
                train_dl, val_dl = dm.prepare_train_loaders(aspect, num_lbls)
                
                # 2. Init & Train
                model = ProteinClassifier(Config.EMBED_DIM, num_lbls).to(Config.DEVICE)
                trainer = Trainer(model, train_dl, val_dl, Config)
                trainer.fit()
                
                # 3. Save
                checkpoint = {
                    'model_state_dict': model.state_dict(),
                    'term_to_idx': dm.term_to_idx,
                    'config': {'num_labels': num_lbls}
                }
                torch.save(checkpoint, save_path)
                print(f"✅ Model saved to: {save_path}")
                
            except Exception as e:
                print(f"❌ Training failed for {aspect}: {e}")
            
            # 4. Cleanup
            if 'model' in locals(): del model
            if 'trainer' in locals(): del trainer
            if 'train_dl' in locals(): del train_dl
            if 'val_dl' in locals(): del val_dl
            gc.collect()
            torch.cuda.empty_cache()
            
        print("\n🎉 TRAINING COMPLETED.")

    # ==========================\n    # PHASE 2: INFERENCE (ENSEMBLE)\n    # ==========================
    else:
        print("🔮 STARTING MULTI-ONTOLOGY INFERENCE PIPELINE...")
        
        # 1. Load Test Data
        dm = CAFA6DataManager(Config)
        test_dl = dm.prepare_test_loader()
        
        all_dfs = [] 
        
        # 2. Loop through models
        for aspect, num_lbls, load_path in aspects_config:
            print(f"\n >>> Processing Aspect: {aspect} <<<")
            
            # Initialize variables to None to prevent NameError in cleanup
            model = None
            trainer = None
            probs = None
            
            try:
                # a. Load Model
                model, term_to_idx = load_specialist_model(load_path, num_lbls)
                
                # b. Predict (DL)
                trainer = Trainer(model, None, None, Config)
                probs = trainer.predict(test_dl)
                
                # c. Propagation
                propagator = GOPropagator(Config.GO_OBO, term_to_idx)
                probs = propagator.propagate(probs)
                
                # d. Create Temp DataFrame
                idx_to_term = {v: k for k, v in term_to_idx.items()}
                rows, cols = np.where(probs > Config.THRESHOLD)
                print(f"   Found {len(rows)} predictions for {aspect}.")
                
                df_temp = pd.DataFrame({
                    'EntryID': [dm.test_ids[i] for i in rows],
                    'term': [idx_to_term[j] for j in cols],
                    'score': probs[rows, cols]
                })
                all_dfs.append(df_temp)
                
            except Exception as e:
                print(f"⚠️ Error processing {aspect}: {e}")
                print(f"   (Check if {load_path} matches the model architecture for {aspect})")
            
            # Cleanup safely
            if model is not None: del model
            if trainer is not None: del trainer
            if probs is not None: del probs
            gc.collect()
            torch.cuda.empty_cache()
            
        # 3. Concatenate
        if len(all_dfs) > 0:
            print("\n🔗 Concatenating predictions from all models...")
            final_df = pd.concat(all_dfs, ignore_index=True)
            print(f"📝 Final submission has {len(final_df)} rows.")
            final_df.to_csv('submission.tsv', sep='\t', header=False, index=False)
            print("✅ DONE! submission.tsv created.")
            print(final_df.head())
        else:
            print("❌ No predictions generated! Check your model paths.")

🔮 STARTING MULTI-ONTOLOGY INFERENCE PIPELINE...
📚 Parsing Aspect (Namespace) from /kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo...
   Mapped 48101 terms to aspects.
📥 Loading Embeddings source...
   Loaded 287001 protein embeddings.
📖 Reading Test Fasta: /kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta
   Constructing Test Matrix...
   Found vectors for 224309/224309 test proteins.

 >>> Processing Aspect: BPO <<<
📥 Loading specialist model from /kaggle/input/multi-ontology/pytorch/default/1/model_BPO.pth...
🔮 Predicting...


  0%|          | 0/1753 [00:00<?, ?it/s]

🌳 Parsing OBO structure from: /kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo
🌊 Starting Score Propagation (Bottom-up)...
   Found 863 parent-child relationships within the 1500 predicted labels.
   Iteration 1 complete.
   Iteration 2 complete.
   Iteration 3 complete.
   Found 12430733 predictions for BPO.

 >>> Processing Aspect: CCO <<<
📥 Loading specialist model from /kaggle/input/multi-ontology/pytorch/default/1/model_CCO.pth...
🔮 Predicting...


  0%|          | 0/1753 [00:00<?, ?it/s]

🌳 Parsing OBO structure from: /kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo
🌊 Starting Score Propagation (Bottom-up)...
   Found 452 parent-child relationships within the 800 predicted labels.
   Iteration 1 complete.
   Iteration 2 complete.
   Iteration 3 complete.
   Found 6731045 predictions for CCO.

 >>> Processing Aspect: MFO <<<
📥 Loading specialist model from /kaggle/input/multi-ontology/pytorch/default/1/model_MFO.pth...
🔮 Predicting...


  0%|          | 0/1753 [00:00<?, ?it/s]

🌳 Parsing OBO structure from: /kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo
🌊 Starting Score Propagation (Bottom-up)...
   Found 598 parent-child relationships within the 800 predicted labels.
   Iteration 1 complete.
   Iteration 2 complete.
   Iteration 3 complete.
   Found 4449028 predictions for MFO.

🔗 Concatenating predictions from all models...
📝 Final submission has 23610806 rows.
✅ DONE! submission.tsv created.
      EntryID        term     score
0  A0A0C5B5G6  GO:0006355  0.029428
1  A0A0C5B5G6  GO:0045893  0.027978
2  A0A0C5B5G6  GO:0006974  0.008850
3  A0A0C5B5G6  GO:0008285  0.005888
4  A0A0C5B5G6  GO:0050832  0.115165


In [14]:
final_df['score'].max()

0.9999929666519165